In [4]:
from transformers import pipeline

classifier = pipeline(
   'zero-shot-classification',
    model="valhalla/distilbart-mnli-12-3",
)

Device set to use cpu


In [10]:
labels = ['clima', 'dolar', 'uf', 'noticias',]
result = classifier('Que clima hace hoy en Santiago?', candidate_labels=labels) 
print(result['labels'][0]) 
result2 = classifier('Que hay de nuevo en Santiago?', candidate_labels=labels)
print(result2['labels'][0])
result3 = classifier('El dolar subio hoy', candidate_labels=labels)
print(result3['labels'][0]) 

clima
noticias
dolar
